<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesion_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilo-vega-169084b1/

---

## Guía: GenBI - Interfaces Conversacionales de Datos con LLM

Este notebook presenta **2 implementaciones prácticas**:

1. **RAG con Gradio** - Chat de documentos con interfaz simple
2. **Text-to SQL y MatplotLib y Seaborn** - Consultas naturales a bases de datos

### Requisitos
- **APIs:** Groq API token
- **GPU:** Opcional para modelos locales
- **Datos:** Documentos PDF/DOCX/TXT/CSV para subir

## Configuración APIs
- **Groq API:**
  1. [Crear token](https://console.groq.com/keys)
  2. En Colab: 🔑 Secrets → Agregar `GROQ_KEY` → Pegar tu token

Cada sección es **independiente** y puede ejecutarse por separado

In [ ]:

#=============================================================================
# 1. RAG CON GRADIO - CHAT DE DOCUMENTOS
#=============================================================================

# Instalaciones necesarias
!pip install gradio groq scikit-learn PyPDF2 python-docx pandas openpyxl -q

import os
import gradio as gr
import pandas as pd
import numpy as np
from groq import Groq
from google.colab import userdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader
from docx import Document

# Configuración global
GROQ_API_KEY = userdata.get('GROQ_KEY')
client = Groq(api_key=GROQ_API_KEY)

# Variables globales del sistema RAG
vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
chunks = []
vectors = None
documents_loaded = False

def process_uploaded_files(files):
    """Procesa archivos subidos y extrae texto"""
    if not files:
        return "No se subieron archivos"

    combined_text = ""
    processed_files = []

    for file in files:
        try:
            file_path = file.name
            filename = os.path.basename(file_path)

            # Procesamiento según tipo de archivo
            if filename.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                    combined_text += text + "\n\n"

            elif filename.endswith('.pdf'):
                reader = PdfReader(file_path)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
                combined_text += text + "\n\n"

            elif filename.endswith('.docx'):
                doc = Document(file_path)
                text = ""
                for paragraph in doc.paragraphs:
                    text += paragraph.text + "\n"
                combined_text += text + "\n\n"

            elif filename.endswith('.csv'):
                df = pd.read_csv(file_path)
                combined_text += df.to_string() + "\n\n"

            elif filename.endswith('.xlsx'):
                df = pd.read_excel(file_path)
                combined_text += df.to_string() + "\n\n"

            processed_files.append(filename)

        except Exception as e:
            continue

    if combined_text.strip():
        # Crear chunks para búsqueda
        create_knowledge_base(combined_text)
        return f"Procesados: {', '.join(processed_files)}\nDocumentos listos para consultas."
    else:
        return "Error procesando archivos"

def create_knowledge_base(text):
    """Crea la base de conocimiento vectorial"""
    global chunks, vectors, vectorizer, documents_loaded

    # Dividir texto en chunks
    words = text.split()
    chunks = []

    chunk_size = 500
    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.strip()) > 100:
            chunks.append(chunk.strip())

    # Crear vectores TF-IDF
    if chunks:
        vectors = vectorizer.fit_transform(chunks)
        documents_loaded = True

def search_documents(query, top_k=3):
    """Busca fragmentos relevantes en los documentos"""
    if not documents_loaded or not chunks:
        return []

    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, vectors)[0]

    # Obtener fragmentos más relevantes
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    relevant_chunks = []
    for idx in top_indices:
        if similarities[idx] > 0.1:
            relevant_chunks.append(chunks[idx])

    return relevant_chunks

def chat_with_documents(message, history):
    """Función principal del chat"""
    if not documents_loaded:
        return "Primero sube documentos para poder consultar"

    # Buscar contexto relevante
    context_chunks = search_documents(message)

    if not context_chunks:
        return "No encontré información relevante sobre tu consulta"

    # Crear contexto para el LLM
    context = "\n\n".join(context_chunks)

    prompt = f"""Basándote únicamente en esta información:

{context}

Pregunta del usuario: {message}

Instrucciones:
- Responde solo con información del contexto proporcionado
- Si no hay información suficiente, menciona que no puedes responder con los documentos disponibles
- Sé claro y conciso
- Responde en español"""

    try:
        response = client.chat.completions.create(
            model="openai/gpt-oss-20b",  # Modelo
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.3
        )
        return response.choices[0].message.content

    except Exception as e:
        return f"Error al procesar la consulta: {str(e)}"

# Crear interfaz Gradio
with gr.Blocks(title="Chat de Documentos", theme=gr.themes.Soft()) as demo:

    gr.Markdown("# Chat de Documentos con RAG")
    gr.Markdown("Sube documentos y haz preguntas sobre su contenido")

    with gr.Row():
        with gr.Column(scale=1):
            # Panel de carga de archivos
            file_upload = gr.File(
                label="Subir Documentos",
                file_count="multiple",
                file_types=[".pdf", ".docx", ".txt", ".csv", ".xlsx"]
            )

            upload_status = gr.Textbox(
                label="Estado",
                value="Esperando documentos...",
                interactive=False
            )

        with gr.Column(scale=2):
            # Interfaz de chat
            chatbot = gr.ChatInterface(
                fn=chat_with_documents,
                title="Asistente de Documentos",
                description="Haz preguntas sobre los documentos subidos",
                examples=[
                    "¿De qué tratan estos documentos?",
                    "Resume los puntos principales",
                    "¿Qué conclusiones se mencionan?"
                ]
            )

    # Conectar eventos
    file_upload.change(
        fn=process_uploaded_files,
        inputs=[file_upload],
        outputs=[upload_status]
    )

# Lanzar la aplicación
if __name__ == "__main__":
    demo.launch(share=True, debug=False)

In [ ]:
# RAG Database Chatbot with Gradio
# Sistema de consultas inteligentes con interfaz de chat y gráficas automáticas

# Instalación de dependencias
!pip install groq sentence-transformers pandas numpy scikit-learn gradio matplotlib seaborn plotly openpyxl -q

import pandas as pd
import numpy as np
import sqlite3
import json
import io
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import userdata
import warnings
warnings.filterwarnings('ignore')

# Configuración global con secretos de Colab
GROQ_API_KEY = userdata.get('GROQ_KEY')
client = Groq(api_key=GROQ_API_KEY)
GROQ_MODEL = "llama-3.1-8b-instant"  # Modelo más estable

# Variables globales del sistema
model = SentenceTransformer('all-MiniLM-L6-v2')
conn = None
table_name = 'data'
column_embeddings = {}
column_info = {}
schema_description = ""
current_data = None

class DatabaseRAG:
    def __init__(self):
        self.conn = None
        self.column_embeddings = {}
        self.column_info = {}
        self.schema_description = ""
        self.current_data = None

    def load_file(self, file_path):
        """Carga archivos CSV, XLSX o JSON y crea base de datos en memoria"""
        try:
            file_extension = file_path.lower().split('.')[-1]

            if file_extension == 'csv':
                df = pd.read_csv(file_path)
            elif file_extension in ['xlsx', 'xls']:
                df = pd.read_excel(file_path)
            elif file_extension == 'json':
                df = pd.read_json(file_path)
            else:
                return False, "Formato no soportado. Use CSV, XLSX o JSON."

            if df.empty:
                return False, "El archivo está vacío."

            # Crear conexión SQLite en memoria
            if self.conn:
                self.conn.close()
            self.conn = sqlite3.connect(':memory:', check_same_thread=False)
            df.to_sql(table_name, self.conn, index=False, if_exists='replace')

            # Guardar datos actuales para gráficas
            self.current_data = df

            # Generar embeddings y descripción del esquema
            self._generate_schema(df)

            return True, f"✅ Archivo cargado: {df.shape[0]} filas, {df.shape[1]} columnas"

        except Exception as e:
            return False, f"Error cargando archivo: {str(e)}"

    def _generate_schema(self, df):
        """Genera esquema y embeddings de columnas"""
        self.column_embeddings.clear()
        self.column_info.clear()
        schema_parts = []

        for col in df.columns:
            # Detectar tipo de columna
            if df[col].dtype in ['int64', 'float64']:
                col_type = 'NUMERIC'
                sample_values = f"rango: {df[col].min():.2f} - {df[col].max():.2f}"
            elif df[col].nunique() <= 20:
                col_type = 'CATEGORICAL'
                unique_vals = list(df[col].unique())
                sample_values = f"valores: {', '.join(map(str, unique_vals[:5]))}"
            else:
                col_type = 'TEXT'
                sample_values = f"texto con {df[col].nunique()} valores únicos"

            desc = f"{col} ({col_type}): {sample_values}"
            self.column_info[col] = {
                'type': col_type,
                'sample_values': sample_values,
                'description': desc
            }

            # Generar embedding
            self.column_embeddings[col] = model.encode([desc])[0]
            schema_parts.append(f"- {col} ({col_type}): {sample_values}")

        self.schema_description = f"""
Tabla: {table_name}
Total de registros: {len(df)}
Columnas disponibles:
""" + "\n".join(schema_parts)

    def find_relevant_columns(self, query, top_k=5):
        """Encuentra columnas relevantes usando búsqueda semántica"""
        query_emb = model.encode([query])[0]
        scores = []
        for col, col_emb in self.column_embeddings.items():
            sim = cosine_similarity([query_emb], [col_emb])[0][0]
            scores.append((col, sim, self.column_info[col]))
        return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

    def generate_sql(self, query):
        """Genera consulta SQL usando LLM - IDÉNTICO AL SCRIPT ORIGINAL"""
        if not self.conn:
            return "SELECT 1"  # Consulta dummy si no hay datos

        relevant_cols = self.find_relevant_columns(query)

        # Construir contexto de columnas relevantes - IGUAL AL ORIGINAL
        cols_context = "\nColumnas más relevantes para la consulta:\n"
        for col, score, info in relevant_cols:
            cols_context += f"- {col} ({info['type']}): {info['sample_values']}\n"

        prompt = f"""Eres un experto en SQL. Genera una consulta SQL para responder la pregunta del usuario.

ESQUEMA DE LA BASE DE DATOS:
{self.schema_description}

{cols_context}

PREGUNTA DEL USUARIO: {query}

INSTRUCCIONES:
1. Usa el nombre de tabla: {table_name}
2. NO uses LIMIT - necesitamos todos los datos
3. Si necesitas agrupar, usa GROUP BY apropiadamente
4. Si necesitas ordenar, usa ORDER BY
5. Para consultas numéricas usa SUM, AVG, COUNT, etc.
6. Responde SOLO con la consulta SQL, sin explicaciones

CONSULTA SQL:"""

        try:
            response = client.chat.completions.create(
                model=GROQ_MODEL,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0.1
            )
            sql_generated = response.choices[0].message.content.strip()

            # Limpiar la respuesta - IGUAL AL ORIGINAL
            sql_generated = sql_generated.replace('```sql', '').replace('```', '').strip()

            return sql_generated
        except Exception as e:
            print(f"Error generando SQL: {e}")
            # Fallback simple - SIN LIMIT
            relevant_col_names = [col for col, _, _ in relevant_cols[:3]]
            return f"SELECT {', '.join(relevant_col_names)} FROM {table_name}"

    def execute_query(self, sql):
        """Ejecuta la consulta SQL en la base de datos - IDÉNTICO AL ORIGINAL"""
        if not self.conn:
            return "Error: Base de datos no inicializada", pd.DataFrame()

        try:
            print(f"Ejecutando SQL: {sql}")
            resultado = pd.read_sql_query(sql, self.conn)
            return sql, resultado
        except Exception as e:
            print(f"Error SQL: {e}")
            # Fallback: mostrar todas las columnas relevantes - SIN LIMIT
            try:
                fallback_sql = f"SELECT * FROM {table_name}"
                resultado = pd.read_sql_query(fallback_sql, self.conn)
                return fallback_sql, resultado
            except Exception as e2:
                print(f"Error en consulta fallback: {e2}")
                return "Error", pd.DataFrame()

    def should_create_chart(self, query):
        """Determina si se debe crear una gráfica basado en la consulta"""
        chart_keywords = [
            'gráfica', 'grafica', 'gráfico', 'grafico', 'chart', 'plot',
            'visualiza', 'muestra', 'dibuja', 'representa',
            'comparar', 'comparación', 'distribución', 'tendencia',
            'ranking', 'top', 'mayor', 'menor', 'evolución'
        ]
        query_lower = query.lower()
        return any(keyword in query_lower for keyword in chart_keywords)

    def create_chart(self, data, query):
        """Crea gráfica automática basada en los datos - SIN LIMITACIONES DE REGISTROS"""
        if data.empty:
            return None

        try:
            # Detectar tipos de columnas
            numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
            categorical_cols = data.select_dtypes(include=['object']).columns.tolist()

            fig = None

            # Lógica de gráficas automáticas - PROCESANDO TODOS LOS DATOS
            if len(numeric_cols) >= 1 and len(categorical_cols) >= 1:
                # Gráfica de barras: categórica vs numérica
                cat_col = categorical_cols[0]
                num_col = numeric_cols[0]

                # Procesar TODOS los datos, solo limitar visualización si hay demasiados valores únicos
                chart_data = data.groupby(cat_col)[num_col].sum().reset_index()

                # Solo mostrar top 20 para legibilidad, pero calculado sobre TODOS los datos
                if chart_data.shape[0] > 20:
                    chart_data = chart_data.nlargest(20, num_col)

                fig = px.bar(
                    chart_data,
                    x=cat_col,
                    y=num_col,
                    title=f"{num_col} por {cat_col} (Top 20 de {data[cat_col].nunique()} categorías)",
                    template="plotly_white"
                )
                fig.update_xaxes(tickangle=45)

            elif len(numeric_cols) >= 2:
                # Scatter plot para dos variables numéricas - TODOS LOS DATOS
                fig = px.scatter(
                    data,
                    x=numeric_cols[0],
                    y=numeric_cols[1],
                    title=f"{numeric_cols[1]} vs {numeric_cols[0]} ({len(data)} registros)",
                    template="plotly_white"
                )

            elif len(numeric_cols) == 1:
                # Histograma para una variable numérica - TODOS LOS DATOS
                fig = px.histogram(
                    data,
                    x=numeric_cols[0],
                    title=f"Distribución de {numeric_cols[0]} ({len(data)} registros)",
                    template="plotly_white"
                )

            elif len(categorical_cols) >= 1:
                # Gráfica de conteo para categóricas - TODOS LOS DATOS
                cat_col = categorical_cols[0]
                value_counts = data[cat_col].value_counts()

                # Solo mostrar top 20 para legibilidad visual
                if len(value_counts) > 20:
                    display_counts = value_counts.head(20)
                    title_suffix = f" (Top 20 de {len(value_counts)} categorías)"
                else:
                    display_counts = value_counts
                    title_suffix = f" ({len(value_counts)} categorías total)"

                fig = px.bar(
                    x=display_counts.index,
                    y=display_counts.values,
                    title=f"Frecuencia de {cat_col}{title_suffix}",
                    template="plotly_white"
                )
                fig.update_xaxes(tickangle=45)

            if fig:
                fig.update_layout(height=400, showlegend=False)
                return fig

        except Exception as e:
            print(f"Error creando gráfica: {e}")

        return None

    def generate_answer(self, query, sql_executed, data):
        """Genera respuesta natural usando LLM con análisis estadístico - IDÉNTICO AL ORIGINAL"""
        if data.empty:
            return "No se encontraron datos para esta consulta."

        # Preparar estadísticas automáticas - IGUAL AL ORIGINAL
        stats_summary = f"Se procesaron {len(data)} registros totales.\n"

        # Agregar estadísticas de columnas numéricas
        numeric_cols = data.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            stats_summary += "\nEstadísticas numéricas:\n"
            for col in numeric_cols[:3]:
                stats_summary += f"- {col}: total={data[col].sum():.2f}, promedio={data[col].mean():.2f}\n"

        # Preparar muestra de datos - IGUAL AL ORIGINAL
        data_sample = data.head(15).to_string(index=False) if len(data) > 15 else data.to_string(index=False)
        if len(data) > 15:
            data_sample += f"\n... (mostrando 15 de {len(data)} registros)"

        prompt = f"""Analiza estos resultados de base de datos y responde la pregunta del usuario de forma clara y profesional.

PREGUNTA ORIGINAL: {query}
SQL EJECUTADO: {sql_executed}

ESTADÍSTICAS:
{stats_summary}

DATOS OBTENIDOS:
{data_sample}

INSTRUCCIONES:
- Responde basándote en los {len(data)} registros procesados
- Da números exactos y estadísticas precisas
- Responde en español de forma profesional y directa
- Si hay rankings, menciona los elementos más importantes
- No repitas el SQL ni la pregunta

RESPUESTA:"""

        try:
            response = client.chat.completions.create(
                model=GROQ_MODEL,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300,
                temperature=0.2
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Error generando respuesta: {e}"

# Instancia global del sistema RAG
rag_system = DatabaseRAG()

def process_file_upload(file):
    """Procesa la carga de archivos"""
    if file is None:
        return "Por favor, sube un archivo CSV, XLSX o JSON.", None

    success, message = rag_system.load_file(file.name)

    if success:
        # Mostrar esquema como bienvenida
        return message + "\n\n" + rag_system.schema_description, gr.update(interactive=True)
    else:
        return message, gr.update(interactive=False)

def chat_response(message, history):
    """Procesa mensajes del chat y genera respuestas"""
    if not rag_system.conn:
        return history + [[message, "Por favor, primero sube un archivo de datos."]]

    if not message.strip():
        return history + [[message, "Por favor, escribe una pregunta."]]

    try:
        # Pipeline RAG completo
        sql = rag_system.generate_sql(message)
        sql_executed, data = rag_system.execute_query(sql)
        respuesta = rag_system.generate_answer(message, sql_executed, data)

        # Formatear respuesta
        full_response = respuesta

        # Actualizar historial
        new_history = history + [[message, full_response]]

        return new_history

    except Exception as e:
        error_msg = f"Error procesando consulta: {str(e)}"
        return history + [[message, error_msg]]

def get_chart_for_last_query(message):
    """Genera gráfica para la consulta actual"""
    if not rag_system.conn or not message.strip():
        return None

    if rag_system.should_create_chart(message):
        try:
            sql = rag_system.generate_sql(message)
            _, data = rag_system.execute_query(sql)
            return rag_system.create_chart(data, message)
        except Exception as e:
            print(f"Error generando gráfica: {e}")
            return None

    return None

# Interfaz Gradio
def create_interface():
    with gr.Blocks(title="RAG Database Chatbot", theme=gr.themes.Soft()) as app:
        gr.Markdown("# 🤖 RAG Database Chatbot")
        gr.Markdown("Sube tu archivo de datos (CSV, XLSX, JSON) y haz preguntas en lenguaje natural.")

        with gr.Row():
            with gr.Column(scale=1):
                file_input = gr.File(
                    label="📁 Cargar Archivo",
                    file_types=[".csv", ".xlsx", ".xls", ".json"]
                )
                file_status = gr.Textbox(
                    label="Estado del Archivo",
                    interactive=False,
                    max_lines=10
                )

            with gr.Column(scale=2):
                chatbot = gr.Chatbot(
                    label="Conversación",
                    height=400,
                    show_copy_button=True
                )

                msg_input = gr.Textbox(
                    label="Tu pregunta",
                    placeholder="Ejemplo: ¿Cuáles son las ventas por región?",
                    interactive=False
                )

                with gr.Row():
                    send_btn = gr.Button("Enviar", variant="primary")
                    clear_btn = gr.Button("Limpiar Chat")

        # Área para gráficas
        chart_output = gr.Plot(label="📊 Gráfica Automática", visible=False)

        # Event handlers - VERSIÓN MEJORADA PARA EVITAR BLOQUEOS
        file_input.change(
            fn=process_file_upload,
            inputs=[file_input],
            outputs=[file_status, msg_input],
            queue=False
        )

        def submit_message(message, history):
            """Maneja el envío de mensajes sin bloqueos"""
            if not message.strip():
                return history, message, None, gr.update(visible=False)

            # Procesar respuesta del chat
            new_history = chat_response(message, history)

            # Generar gráfica si es necesario
            chart = get_chart_for_last_query(message)
            chart_visible = chart is not None

            # Limpiar input y mostrar resultados
            return new_history, "", chart, gr.update(visible=chart_visible)

        # Conectar eventos con mejor manejo
        send_btn.click(
            fn=submit_message,
            inputs=[msg_input, chatbot],
            outputs=[chatbot, msg_input, chart_output, chart_output],
            queue=True
        )

        msg_input.submit(
            fn=submit_message,
            inputs=[msg_input, chatbot],
            outputs=[chatbot, msg_input, chart_output, chart_output],
            queue=True
        )

        def clear_chat():
            """Limpia el chat y oculta gráficas"""
            return [], None, gr.update(visible=False)

        clear_btn.click(
            fn=clear_chat,
            outputs=[chatbot, chart_output, chart_output],
            queue=False
        )

        # Ejemplos
        gr.Examples(
            examples=[
                "¿Cuáles son las 10 ciudades con mayores ventas?",
                "Muestra la distribución de edades",
                "¿Cuál es el promedio de ingresos por departamento?",
                "Grafica las ventas por mes",
                "¿Qué productos tienen mejor rendimiento?"
            ],
            inputs=msg_input
        )

    return app

if __name__ == "__main__":
    # Crear y lanzar la aplicación
    app = create_interface()
    app.launch(
        share=True,
        show_error=True
    )